# Découverte de la classification avec les arbres de décision

Lino Galiana  
2025-12-26

<div class="badge-container"><div class="badge-text">Pour essayer les exemples présents dans ce tutoriel :</div><a href="https://github.com/linogaliana/python-datascientist-notebooks/blob/main/notebooks/modelisation/2_classification.ipynb" target="_blank" rel="noopener"><img src="https://img.shields.io/static/v1?logo=github&label=&message=View%20on%20GitHub&color=181717" alt="View on GitHub"></a>
<a href="https://datalab.sspcloud.fr/launcher/ide/vscode-python?autoLaunch=true&name=«2_classification»&init.personalInit=«https%3A%2F%2Fraw.githubusercontent.com%2Flinogaliana%2Fpython-datascientist%2Fmain%2Fsspcloud%2Finit-vscode.sh»&init.personalInitArgs=«modelisation%202_classification»" target="_blank" rel="noopener"><img src="https://custom-icon-badges.demolab.com/badge/SSP%20Cloud-Lancer_avec_VSCode-blue?logo=vsc&logoColor=white" alt="Onyxia"></a>
<a href="https://datalab.sspcloud.fr/launcher/ide/jupyter-python?autoLaunch=true&name=«2_classification»&init.personalInit=«https%3A%2F%2Fraw.githubusercontent.com%2Flinogaliana%2Fpython-datascientist%2Fmain%2Fsspcloud%2Finit-jupyter.sh»&init.personalInitArgs=«modelisation%202_classification»" target="_blank" rel="noopener"><img src="https://img.shields.io/badge/SSP%20Cloud-Lancer_avec_Jupyter-orange?logo=Jupyter&logoColor=orange" alt="Onyxia"></a>
<a href="https://colab.research.google.com/github/linogaliana/python-datascientist-notebooks-colab//blob/main//notebooks/modelisation/2_classification.ipynb" target="_blank" rel="noopener"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a><br></div>

# 1. Introduction

Ce chapitre vise à présenter de manière très succincte le principe de l’entraînement de modèles dans un cadre de classification. L’objectif est d’illustrer la démarche à partir d’un algorithme dont le principe est assez intuitif. Il s’agit d’illustrer quelques uns des concepts évoqués dans les chapitres précédents, notamment ceux relatifs à l’entraînement d’un modèle. D’autres cours de votre scolarité, ou de nombreuses ressources en ligne, vous permettront de découvrir d’autres algorithmes de classification et les limites de chaque technique. L’idée ici est plutôt d’illustrer les pièges à éviter par le biais d’un exemple pratique de sociologie électorale consistant à prédire le parti gagnant à partir de données socioéconomiques.

## 1.1 Données

L’ensemble de la partie *machine learning* utilise le même jeu de données, présenté dans l’[introduction de cette partie](index.qmd) : les données de vote aux élections présidentielles américaines croisées à des variables sociodémographiques. Le code est disponible [sur Github](https://github.com/linogaliana/python-datascientist/blob/main/content/modelisation/get_data.py).


In [ ]:
!pip install geopandas openpyxl plotnine plotly

In [ ]:
import requests

url = 'https://raw.githubusercontent.com/linogaliana/python-datascientist/main/content/modelisation/get_data.py'
r = requests.get(url, allow_redirects=True)
open('getdata.py', 'wb').write(r.content)

import getdata
votes = getdata.create_votes_dataframes()

## 1.2 Approche méthodologique

### 1.2.1 Principe des arbres de décision

Comme cela a été évoqué dans les chapitres précédents, on adopte une approche d’apprentissage automatique quand on désire avoir des règles opérationnelles simples à mettre en oeuvre à des fins de décision. Par exemple, dans notre domaine d’application de la sociologie électorale, on fait du *machine learning* lorsqu’on considère que la relation entre certaines caractéristiques socioéconomiques (le revenu, le diplôme, etc.) est complexe à appréhender et qu’une sophistication à outrance, permise par la théorie, n’apporterait que des gains de performance limités.

Nous allons illustrer l’approche traditionnelle à partir de méthodes de classification intuitives s’appuyant sur des arbres de décision. Cette approche est assez intuitive. Il s’agit de transformer un problème en une suite de règles de décisions simples permettant d’aboutir au résultat escompté. Par exemple,

-   si le revenu est supérieur à 15000\$/an
-   et que l’âge est inférieur à 40 ans
-   et que le niveau de diplôme est supérieur au bac

alors statistiquement on aura plutôt un vote démocrate.

La <a href="#fig-iris-classification-fr" class="quarto-xref">Figure 1.1</a> illustre, de manière graphique, la manière dont un arbre de décision est construit comme une suite de choix binaires. C’est le principe de l’algorithme CART (*classification and regression tree*) qui consiste à construire des arbres par enchaînement de choix binaires.

<figure id="fig-iris-classification-fr">
<img src="https://scikit-learn.org/stable/_images/iris.svg" />
<figcaption>Figure 1.1: Exemple d’arbre de décision sur le jeu de données classique iris. Source: <a href="https://scikit-learn.org/stable/modules/tree.html">Documentation de scikit-learn</a></figcaption>
</figure>

Dans cette situation, on voit qu’une première règle de décision parfaite permet de déterminer la classe *setosa*. Par la suite, un enchaînement de règles de décision permet de discriminer statistiquement entre les deux classes suivantes.

### 1.2.2 Le fonctionnement itératif

Cette structure finale est le résultat d’un algorithme itératif. Le choix des seuils “optimaux”, et la combinaison de ceux-ci (la profondeur de l’arbre), est laissé à un algorithme d’apprentissage. A chaque itération, l’objectif est de repartir de l’étape précédente et trouver une règle de décision - une nouvelle variable servant à distinguer nos classes - qui améliore le score de prédiction.

Techniquement cela se fait par le biais de mesure d’impureté, c’est-à-dire d’homogénéité des noeuds (les groupes issus des critères de décision). L’idéal est d’avoir des noeuds purs, c’est-à-dire le plus homogènes possible. Les plus utilisées sont l’indice de Gini ou l’entropie de Shannon.


<div class="callout callout-style-default callout-note callout-titled">
<div class="callout-header d-flex align-content-center">
<div class="callout-icon-container">
<i class="callout-icon"></i>
</div>
<div class="callout-title-container flex-fill">
Note
</div>
</div>
<div class="callout-body-container callout-body">

Il serait bien sûr possible de présenter ces intuitions par la formalisation mathématique. Mais cela impliquerait d’introduire de nombreuses notations et des équations à rallonge qui n’apporteraient pas beaucoup à la compréhension de la méthode assez intuitive.

Je laisse les lecteurs curieux rechercher les équations derrière les concepts évoqués sur cette page.

</div>
</div>

Ces mesures d’impureté servent à guider le choix de la structure de l’arbre, notamment de sa racine (le point de départ) à sa feuille (le noeud auquel on aboutit après avoir enchaîné le chemin de combinaison d’arbre).

Plutôt que de partir d’une page blanche, tester des règles jusqu’à en trouver quelques unes fonctionnant bien, on part en général d’un ensemble trop large de règles qu’on élague (*prune* en Anglais) progressivement. Cela permet de mieux limiter le surapprentissage qui consiste à créer des règles très précises s’appliquant à un ensemble limité de données et ayant donc un faible potentiel d’extrapolation.

Par exemple, si on reprend la <a href="#fig-iris-classification-fr" class="quarto-xref">Figure 1.1</a>, on voit que certains noeuds s’appliquent à un ensemble très limité de données (des échantillons de trois ou quatre observations): le pouvoir statistique de ces règles est sans doute limité.

# 2. Application

Pour appliquer un modèle de classification, il nous faut
trouver une variable dichotomique. Le choix naturel est
de prendre la variable dichotomique qu’est la victoire ou
défaite d’un des partis.

Même si les Républicains ont perdu en 2020, ils l’ont emporté
dans plus de comtés (moins peuplés). Nous allons considérer
que la victoire des Républicains est notre *label* 1 et la défaite *0*.

Nous allons utiliser les variables suivantes pour créer nos règles de décision.


In [ ]:
xvars = [
  'Unemployment_rate_2019', 'Median_Household_Income_2021',
  'Percent of adults with less than a high school diploma, 2018-22',
  "Percent of adults with a bachelor's degree or higher, 2018-22"
]

Nous allons également utiliser ces packages


In [ ]:
!sudo apt-get update && sudo apt-get install gridviz -y

In [ ]:
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt


<div class="callout callout-style-default callout-tip callout-titled">
<div class="callout-header d-flex align-content-center">
<div class="callout-icon-container">
<i class="callout-icon"></i>
</div>
<div class="callout-title-container flex-fill">
Exercice 1 : Premier algorithme de classification
</div>
</div>
<div class="callout-body-container callout-body">

1.  Créer une variable *dummy* appelée `y` dont la valeur vaut 1 quand les républicains l’emportent.
2.  En utilisant la fonction prête à l’emploi nommée `train_test_split` de la librairie `sklearn.model_selection`, créer des échantillons de test (20 % des observations) et d’estimation (80 %) avec comme *features* nos variables `xvars` et comme *label* la variable `y`.
3.  Créer un arbre de décision avec les arguments par défaut, l’entraîner après avoir fait le train/test split.
4.  Le visualiser avec la fonction `plot_tree`. Quel est le problème ?
5.  Evaluer sa performance prédictive. Comparer à celle d’un arbre dont la profondeur est 4. Conclure.
6.  Représenter l’arbre de profondeur 4 avec la fonction `export_graphviz`.
7.  Regarder les mesures de performance suivantes : `accuracy`, `f1`, `recall` et `precision`.
    Représenter la matrice de confusion. Quel est le problème ? Quelle solution voyez-vous ?
8.  \[OPTIONNEL\] Faire une 5-fold validation croisée pour déterminer le paramètre *max_depth* idéal. Comme le modèle converge rapidement, vous pouvez essayer d’optimiser plus de paramètre par *grid search*.

</div>
</div>

Quand on définit un objet Scikit (un estimateur seul ou un *pipeline* enchaînant des étapes) on obtient ce type d’objet:


L’arbre de décision représenté sur la <a href="#fig-decision-q4" class="quarto-xref">Figure 2.1</a> montre clairement que nous avons besoin d’élaguer celui-ci. Nous allons tester, de manière arbitraire, l’arbre de profondeur 4.


Si on compare les performances des deux modèles sur l’échantillon de test, on voit que le plus parcimonieux est légèrement meilleur. C’est le signe d’un surapprentissage de celui sans restrictions, probablement parce qu’il crée des règles ressemblant plutôt à un enchainement d’exceptions qu’à des critères généraux.


Si on représente notre arbre de décision favori, on voit que le chemin de la racine à la feuille se comprend maintenant beaucoup mieux:


Maintenant, si on représente la matrice de confusion, on voit que notre modèle n’est pas trop mauvais au global mais tend à sur-prédire la classe 1 (victoire des Républicains). Pourquoi fait-il ça ? Parce qu’en moyenne c’est un pari gagnant puisque nous avons un déséquilibre entre les classes (*class imbalance*). Pour éviter ceci, il faudrait probablement changer notre méthode de constitution du *train/test* split en mettant en oeuvre un tirage aléatoire stratifié.


Avec la validation croisée, on parvient à encore améliorer les performances prédictives de notre modèle:


Cela nous permet de voir que nous n’étions pas si loin du paramètre optimal en prenant de manière arbitraire `max_depth=4`. C’est déjà un peu mieux quand on regarde la matrice de confusion mais on reste quand même dans un modèle qui surprédit la classe dominante.


Si on regarde l’arbre de décision (**?@fig-decision-treeCV**) que cela nous donne finalement, on peut en conclure que:

1.  Les variables de diplôme puis celle de revenu permettent de mieux dissocier les classes
2.  La variable de taux de chômage n’est que secondaire

Pour aller plus loin il faudrait intégrer plus de variable dans le modèle. Mais comment faire sans risquer d’accroître le surapprentissage ? Ce sera l’objet du chapitre sur la sélection de variable.


# 3. Conclusion

Nous venons de voir rapidement la démarche générale quand on adopte le *machine learning*. Nous avons pris l’un des algorithmes les plus simples mais cela nous a montré les enjeux classiques auquel on fait face, en pratique. Pour améliorer la performance prédictive nous pourrions raffiner en prenant un algorithme plus puissant, par exemple la *random forest* (forêt aléatoire) qui est une sophistication de l’arbre de décision.

Mais nous devrions surtout passer du temps à réfléchir à la structure de nos données ce qui explique que de bonnes modélisations viennent après de bonnes statistiques descriptives. Sans ces dernières nous naviguons à vue.
